In [1]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [2]:
# MPC import
import numpy as np
from LinearMPC_template.MPCVelControl import MPCVelControl
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [12]:
Ts = 0.05
sim_time = 7
H = 5

# [w_x, w_y, w_z, alpha, beta, gamma, v_x, v_y, v_z, x, y, z]

# Test Case 1: Velocity Perturbation (Start at 5 m/s in X)
x0_x = np.array([0., 0., 0.,    # Angular velocity (rad/s)
                 0., 0., 0.,    # Euler angles (rad)
                 5., 0., 0.,    # Velocity (m/s) <--- v_x = 5
                 0., 0., 0.])   # Position (m)

# Test Case 2: Roll Perturbation (Start at 30 degrees roll)
x0_roll = np.array([0., 0., 0.,
                    0., 0., np.deg2rad(30), # Gamma = 30 deg
                    0., 0., 0.,
                    0., 0., 0.])

# Test Case 3: Combined (5 m/s in all axes)
x0_all = np.array([0., 0., 0.,
                   0., 0., 0.,
                   5., 5., 5.,  # v_x, v_y, v_z = 5
                   0., 0., 0.])

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0_all, method="linear"
)

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol);

Iteration 1... not yet converged

Iteration 2... not yet converged

Iteration 3... not yet converged

Iteration 4... not yet converged

Iteration 5... not yet converged



c:\Users\Vince\anaconda3\envs\mpc2025\Lib\site-packages\cvxpy\problems\problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Iteration 6... not yet converged

Maximum invariant set successfully computed after 7 iterations.
Iteration 1... not yet converged

Iteration 2... not yet converged

Iteration 3... not yet converged

Iteration 4... not yet converged

Iteration 5... not yet converged

Iteration 6... not yet converged

Maximum invariant set successfully computed after 7 iterations.
Maximum invariant set successfully computed after 1 iterations.
Iteration 1... not yet converged

Iteration 2... not yet converged

Iteration 3... not yet converged

Iteration 4... not yet converged

Iteration 5... not yet converged

Iteration 6... not yet converged

Iteration 7... not yet converged

Iteration 8... not yet converged

Iteration 9... not yet converged

Iteration 10... not yet converged

Iteration 11... not yet converged

Iteration 12... not yet converged

Iteration 13... not yet converged

Iteration 14... not yet converged

Iteration 15... not yet converged

Iteration 16... not yet converged

Iteration 17... not

AppLayout(children=(HBox(children=(Play(value=0, description='Press play', max=139, step=2), IntSlider(value=0…